<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/Bert_embedding_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification


  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 51.9MB/s]
 41% 5.00M/12.1M [00:00<00:00, 30.3MB/s]
100% 12.1M/12.1M [00:00<00:00, 58.8MB/s]
 95% 259M/273M [00:02<00:00, 150MB/s]
100% 273M/273M [00:02<00:00, 120MB/s]


In [4]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325
Receiving objects: 100% (325/325), 234.49 KiB | 648.00 KiB/s, done.
Resolving deltas: 100% (186/186), done.


In [5]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip the file
!unzip uncased_L-12_H-768_A-12.zip

--2019-04-30 12:54:20--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 2a00:1450:400c:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  60.4MB/s    in 6.8s    

2019-04-30 12:54:27 (57.1 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

unzip:  cannot find or open the, the.zip or the.ZIP.
Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: unca

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import numpy as np 
train = pd.read_csv("train.csv.zip")
test = pd.read_csv("test.csv.zip")

df_bert = pd.DataFrame({'user_id':train['id'],
            'label':np.where(train.target > .5,1,0),
            'alpha':['a']*train.shape[0],
            'text':train["comment_text"].replace(r'\n',' ',regex=True)})

df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

# Creating test dataframe according to BERT
df_bert_test = pd.DataFrame({'User_ID':test['id'],
                 'text':test['comment_text'].replace(r'\n',' ',regex=True)})

In [0]:
df_bert_train.to_csv('data/train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('data/dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('data/test.tsv', sep='\t', index=False, header=True)

In [0]:
!python bert/run_classifier.py \
--task_name=cola \
--do_train=true \
--do_eval=true \
--do_predict=true \
--data_dir=data \
--vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
--bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
--init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
--max_seq_length=400 \
--train_batch_size=32 \
--learning_rate=2e-5 \
--num_train_epochs=1.0 \
--output_dir=bert_output \
--save_checkpoints_steps=9999999

BERT feature extraction

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np 
import gc
from tqdm import tqdm
train = pd.read_csv("train.csv.zip")['comment_text'].replace(r'\n', ' ', regex=True)
batch_size = 512
n_batch = len(train) // batch_size
gc.collect()

7

In [0]:
for batch in tqdm(range(2878, n_batch)):
    batch_start = batch_size * batch
    batch_end = batch_size * (batch+1)
    train[batch_start:batch_end].to_csv('input.txt', header=False, index=False)
    tmp_matrix = np.zeros((batch_size, 768), dtype='float16')
    
    !python bert/extract_features.py \
    --input_file=input.txt \
    --output_file=output.jsonl \
    --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
    --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
    --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
    --layers=-1 \
    --max_seq_length=220 \
    --batch_size=64
    
    bert_output = pd.read_json("output.jsonl", lines=True)

    for i in range(batch_size):
        tmp = np.zeros(768, )
        voc = len(bert_output['features'][i])-2
        for j in range(1, voc+1):
            tmp += np.asarray(bert_output['features'][i][j]['layers'][0]['values'], dtype='float16') / voc
        tmp_matrix[i] = tmp
#     if batch > 0:
#         tmp_matrix = np.vstack((tmp_matrix, np.load('drive/My Drive/Data/bert_matrix.npy')))
    np.save('drive/My Drive/Data/parts/bert_matrix_'+str(batch), tmp_matrix)
    del tmp_matrix, bert_output
    gc.collect()
    

INFO:tensorflow:prediction_loop marked as finished
INFO:tensorflow:prediction_loop marked as finished


  0%|          | 1/647 [01:23<14:54:57, 83.12s/it]

In [0]:
# from bert_embedding import BertEmbedding
# import mxnet as mx
# ctx = mx.gpu(0)
bert = BertEmbedding(batch_size=64)

In [0]:
for batch in tqdm(range(n_batch)):
    batch_start = batch_size * batch
    batch_end = batch_size * (batch+1)
    batch_file = train[batch_start:batch_end].tolist()
    tmp_matrix = np.zeros((batch_size, 768), dtype='float16')
    for i, matrix in enumerate(bert(batch_file)):
      tmp_matrix[i] = np.mean(np.asarray(matrix[1]), 0)
    if batch > 0:
      tmp_matrix = np.vstack((tmp_matrix, np.load('drive/My Drive/Data/bert_matrix.npy')))
    np.save('drive/My Drive/Data/bert_matrix', tmp_matrix)
    del tmp_matrix, matrix, batch_file
    gc.collect()

In [0]:
# import numpy as np
a = np.load('drive/My Drive/Data/bert_matrix.npy')
a.shape[0] / 512

361.0